In [5]:
from google.colab import drive
drive.mount('/content/gdrive/')

KeyboardInterrupt: ignored

In [2]:
!pip install python_speech_features

  Created wheel for python-speech-features: filename=python_speech_features-0.6-cp37-none-any.whl size=5888 sha256=7cf1e09bd75b1add2503cd816ee0877f127ffc69e7808aaca4c9b6dfa6e97d67
  Stored in directory: /root/.cache/pip/wheels/3c/42/7c/f60e9d1b40015cd69b213ad90f7c18a9264cd745b9888134be
Successfully built python-speech-features


In [3]:
# Import libraries

from python_speech_features import mfcc
import scipy.io.wavfile as wav
import numpy as np
from tempfile import TemporaryFile
import os
import pickle
import random 
import operator
import math

from sklearn import svm
import sklearn.model_selection as model_selection
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score

In [4]:
# Extract features from the dataset and dump these features into a binary .dat file “my.dat”:

directory = '/content/gdrive/MyDrive/genres'
f = open("mymcsvm.dat", 'wb')
i = 0
y = []
for root, folders, files in os.walk(directory):
  for folder in folders:
    print('Reading folder: ', folder)
    i += 1
    if i == 11:
        break
    for file in os.listdir(directory + "/" + folder):
        (rate, sig) = wav.read(directory + "/" + folder + "/" + file)
        mfcc_feat = mfcc(sig, rate, winlen = 0.020, appendEnergy = False)
        covariance = np.cov(np.matrix.transpose(mfcc_feat))
        covariance.flatten()
        mean_matrix = mfcc_feat.mean(0)
        feature = np.concatenate((mean_matrix, covariance), axis=None)
        y.append(i)
        pickle.dump(feature, f)
        
f.close()

FileNotFoundError: ignored

In [ ]:
# Read the dataset

X = []
with open("mymcsvm.dat", 'rb') as f:
    while True:
        try:
            X.append(pickle.load(f))
        except EOFError:
            f.close()
            break

In [ ]:
# Split train and test dataset

X_train, X_test, y_train, y_test = model_selection.train_test_split(X, y, train_size=0.80, test_size=0.20, random_state=101)

In [ ]:
# Create Linear and Polynomial objects from SVM to create two different classifier

linear = svm.SVC(kernel='linear', gamma=0.5, C = 1).fit(X_train, y_train)
poly = svm.SVC(kernel='poly', degree = 1, C = 1).fit(X_train, y_train)

In [ ]:
# Calculate the effeciency of two models

poly_pred = poly.predict(X_test)
linear_pred = linear.predict(X_test)

In [ ]:
# Result of Polynomial kernel

poly_accuracy = accuracy_score(y_test, poly_pred)
poly_f1 = f1_score(y_test, poly_pred, average='weighted')
print('Accuracy (Polynomial Kernel): ', "%.2f" % (poly_accuracy*100))
print('F1 (Polynomial Kernel): ', "%.2f" % (poly_f1*100))

In [ ]:
# Result of Linear kernel

linear_accuracy = accuracy_score(y_test, linear_pred)
linear_f1 = f1_score(y_test, linear_pred, average='weighted')
print('Accuracy (Linear Kernel): ', "%.2f" % (linear_accuracy*100))
print('F1 (Linear Kernel): ', "%.2f" % (linear_f1*100))